In [1]:
shuffle_data_file = "../temp/out/dev_matched_shuffled.csv"
shuffle_data_pred_file = "../temp/out/dev_matched_shuffle_revese_finetune_predict.txt"
shuffle_data_pred_file = "../temp/out/dev_matched_shuffled_bert_base_finetune_predict.txt"



data_file = "../temp/out/dev_matched.csv"
data_pred_file = "../temp/out/dev_matched_reverse_finetune_predict.txt"
data_pred_file = "../temp/out/dev_matched_bert_base_finetune_predict.txt"


In [2]:
import pandas as pd
pd.set_option('max_colwidth', 0)

In [3]:
def load_df(raw_file, pred_file):

    df = pd.read_csv(raw_file, quotechar='"')
    df["prediction"] = pd.read_csv(pred_file, delimiter="\t")["prediction"]
    df = df.drop(columns = ["sentence1_binary_parse","sentence2_binary_parse", "sentence1_parse", "sentence2_parse"])
    return df

In [4]:
df_shuffle =  load_df(shuffle_data_file, shuffle_data_pred_file)
df_normal =  load_df(data_file, data_pred_file)


In [5]:
df_normal.shape

(9714, 13)

In [6]:
df_normal.query("index == 9713")

,index,promptID,pairID,genre,sentence1,sentence2,label1,label2,label3,label4,label5,label,prediction
9612,9713,88813,88813n,fiction,"120 ""You do not think I ought to go to the police?""",Will the police be able to help?,neutral,neutral,entailment,neutral,neutral,001_neutral,001_neutral


In [7]:
df_normal.head()

,index,promptID,pairID,genre,sentence1,sentence2,label1,label2,label3,label4,label5,label,prediction
0,0,63735,63735n,slate,The new rights are nice enough,Everyone really likes the newest benefits,neutral,entailment,neutral,neutral,neutral,001_neutral,001_neutral
1,1,91383,91383c,government,This site includes a list of all award winners and a searchable database of Government Executive articles.,The Government Executive articles housed on the website are not able to be searched.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction
2,2,755,755e,telephone,uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him,"I like him for the most part, but would still enjoy seeing someone beat him.",entailment,entailment,entailment,entailment,entailment,000_entailment,000_entailment
3,3,78013,78013c,telephone,yeah i i think my favorite restaurant is always been the one closest you know the closest as long as it's it meets the minimum criteria you know of good food,My favorite restaurants are always at least a hundred miles away from my house.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction
4,4,96377,96377c,telephone,i don't know um do you do a lot of camping,I know exactly.,contradiction,contradiction,contradiction,contradiction,contradiction,002_contradiction,002_contradiction


In [9]:
df_normal.shape

(9714, 13)

In [10]:
df_shuffle[["sentence1", "sentence2"]].sample(n=3)

,sentence1,sentence2
6053,confused. I'm,don't understand. I
2418,river. need to able also We will bodied men the help us any spike,will spike the us to need We men river. help disabled
622,"and Aid committee unanimous Compatibility Committee, Order, recommendations the Report in of Federal The Communications an Rulemaking advisory the by 1995. Negotiated in Commission large part, established adopts the Hearing",product Federal Compatibility Rulemaking Aid is the Negotiated Committee Communications The Hearing of Commission. a


In [11]:
df_shuffle["is_normal_correct"] = df_shuffle["index"].isin( df_normal.query("label == prediction")["index"])

In [12]:
df_shuffle.query("label != prediction and label != '001_neutral' and is_normal_correct ==True ").shape

(940, 14)

In [13]:
df_shuffle.query("label != prediction and label != '001_neutral' and is_normal_correct ==True ")\
    [["sentence1", "sentence2"]]

,sentence1,sentence2
11,Hinduism. The rigid new bhakti hitherto ritual Tamils the to of warmth of brought a the Brahmanic movement,froze bhakti The ritual the Tamils' warm of Hinduism. movement previously
13,"and your friends You are not welcome Severn. said here,",Severn said were not people there. welcome the
25,that's it how is. Sorry but,are is are about apologies there how This and it. no things
45,file their uh on from two send uh had guys to were but road them final actually in they when the home exams from uh their thousand these miles and,exams their filed home. men midterm These from
58,"a its heightened possibilities. a has and She turns, its has to tried life hollow get exchanged comprehend all life, for",hollow a life. live chose to has She
...,...,...
9650,"Minoan pumice civilization. and of a eruption not whole Akrotiri b.c. destroyed the , volcanic under but 1500 massive at ash Santorini only feet Around",The volcano has years. lain for dormant thousand four the past
9668,i camping a of know you do don't lot do um,you Do alot? camp
9673,"as blamed until paying is to globalization's official excesses we Rightly its America be or prime head and start mover seen and them. will for wrongly, cheerleader attention",role globalization in important with it movement America's not. we agree is the whether or
9676,hillsides; and many into selling into homes some shops the art converted been are have collectibles. There galleries built,hillside in and selling the have collectibles. shops homes of been converted art galleries the All into


In [14]:
df_shuffle.query("label == prediction and label != '001_neutral' and is_normal_correct ==True ")\
    [["sentence1", "sentence2"]]

,sentence1,sentence2
1,a This and database winners articles. a all award of searchable list includes Executive of site Government,to searched. Government on be are Executive the not website The housed able articles
2,somebody about at see i know the to uh times him sometimes beat him have i don't i uh emotions like i but mixed i same love him,"most still him. like but him for would seeing enjoy I part, beat the someone"
3,as criteria it's know the closest my one favorite food you always think as restaurant yeah good been minimum is long it i the of the i meets closest you know,hundred away My my favorite at a always miles from restaurants least house. are
4,know do of um do a camping lot don't i you,know I exactly.
5,out that got would we're before to well we that would of wish here they i it's little space left that end landfill so be have really the be this help and going do decade a going run to,We plenty have of the in space landfill.
...,...,...
9708,"Second, rate not If entirely. of the only we'd any goal it reducing were, sex worth the social in HIV is event transmission outlaw",HIV problems. No one about social or other cares any
9710,in by malfunction space. were pagers the United knocked out of percent satellite a States Eighty in,by Pagers States the in United the satellite were unaffected malfunction.
9711,"the videotapes, the and entities complying in issue to conduct aid with hold FDA guidance will manuals rule. and Finally, workshops, teleconferences small",is set to workshops. FDA The conduct
9712,"Soleil's that of 1.5 in water. an The million troupe, from aquatic latest liters) million international utilizes gallons dazzles acclaimed O the du environment (6.8 Cirque",an Cirque international troupe. is du Soleil


In [15]:
df_shuffle.query("label == prediction and label != '001_neutral' and is_normal_correct ==True ").shape, df_shuffle.query("label != '001_neutral' and is_normal_correct ==True ").shape

((4754, 14), (5694, 14))

In [16]:
df_shuffle.query("label != '001_neutral'")["is_normal_correct"].value_counts()

True     5694
False    926 
Name: is_normal_correct, dtype: int64

In [17]:
df_shuffle["is_normal_correct"].value_counts()

True     8165
False    1549
Name: is_normal_correct, dtype: int64

In [18]:
(df_normal["label"]==df_normal["prediction"]).value_counts()

True     8165
False    1549
dtype: int64